In [2]:
%load_ext autoreload
%autoreload 2

import os
from pathlib import Path
import shutil
from dotenv import load_dotenv

from collab_data.file_utils import expand_path, get_project_root
from collab_data.gcs_utils import GCSClient

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Grab API keys

In [3]:
# Load environment variables from .env file
load_dotenv()

gerbils_key = os.environ.get("COLLAB_GERBILS_KEY")
data_key = os.environ.get("COLLAB_DATA_KEY")

print(f"Gerbils key: {gerbils_key}")
print(f"Data key: {data_key}")

Gerbils key: /workspace/api-keys/collab-gerbils-444419-2c6aa9a47bb5.json
Data key: /workspace/api-keys/collab-data-463313-c340ad86b28e.json


### Grab current project data

In [4]:
CURRENT_PROJECT = "COLLAB_DATA"
PROJECT_KEY = Path(os.environ.get(f"{CURRENT_PROJECT}_KEY"))
PROJECT_ID = "-".join(PROJECT_KEY.stem.split("-")[:-1])

# Connect to GCS
gcs_client = GCSClient(
    project_id=PROJECT_ID,
    credentials_path=expand_path(PROJECT_KEY.as_posix(), get_project_root()),
)

2025-07-30 17:48:37.397 | INFO     | collab_data.gcs_utils:__init__:34 - Using credentials from /workspace/api-keys/collab-data-463313-c340ad86b28e.json


2025-07-30 17:48:37.490 | INFO     | collab_data.gcs_utils:__init__:40 - Using project collab-data-463313


### Find current folders

In [5]:
all_buckets = gcs_client.list_buckets()
print(f"Available buckets: {all_buckets}")

Available buckets: ['fieldwork_curated', 'fieldwork_processed', 'fieldwork_storage_old', 'roboflow_model']


### Start with the curated

In [ ]:
curated_data_dir = 'fieldwork_processed'

# Find all files in the curated fieldwork
curated_files = gcs_client.glob(f"{curated_data_dir}/*")
curated_dir = curated_files[1]


# file_path = gcs_client.glob(f"{all_buckets[-1]}/field/*final*")[0]

# # Create a local path for downloading
# local_data_path = Path("./data") / Path(file_path).name

# # Download the file
# gcs_client.gcs.get_file(
#     rpath=file_path,
#     lpath=local_data_path.as_posix(),
# )

#### Create a directory and push to gcloud

In [ ]:
# gcs_prefix = f"{BUCKET_NAME}/{SESSION_DIR.name}"
session_dir = "/workspace/fieldwork-data/birds/2024-05-18/"

for root, _, files in os.walk(session_dir):
    for file in files:
        local_path = Path(root) / file
        relative_path = local_path.relative_to(session_dir)
        gcs_path = f"{curated_dir}/{relative_path.as_posix()}"
        # print (relative_path)
        # full_path = session_dir + relative_path
        gcs_client.upload_file(str(local_path), gcs_path)

# # 🔍 List contents to verify
# print("\nUploaded GCS paths:\n")
# for path in gcs_client.glob(f"{curated_dir}/**"):
#     print(path)

2025-07-28 18:48:20.614 | INFO     | collab_data.gcs_utils:upload_file:179 - Uploading file /workspace/fieldwork-data/birds/2024-05-18/environment/C0065/rade-features/mesh/mesh_features.pt to fieldwork_processed/2024_05_18-session_0001/environment/C0065/rade-features/mesh/mesh_features.pt.


2025-07-28 18:48:24.440 | INFO     | collab_data.gcs_utils:upload_file:181 - Uploaded file /workspace/fieldwork-data/birds/2024-05-18/environment/C0065/rade-features/mesh/mesh_features.pt to fieldwork_processed/2024_05_18-session_0001/environment/C0065/rade-features/mesh/mesh_features.pt.
2025-07-28 18:48:24.445 | INFO     | collab_data.gcs_utils:upload_file:179 - Uploading file /workspace/fieldwork-data/birds/2024-05-18/environment/C0065/rade-features/mesh/Open3dTSDFfusion_mesh.ply to fieldwork_processed/2024_05_18-session_0001/environment/C0065/rade-features/mesh/Open3dTSDFfusion_mesh.ply.
2025-07-28 18:48:27.884 | INFO     | collab_data.gcs_utils:upload_file:181 - Uploaded file /workspace/fieldwork-data/birds/2024-05-18/environment/C0065/rade-features/mesh/Open3dTSDFfusion_mesh.ply to fieldwork_processed/2024_05_18-session_0001/environment/C0065/rade-features/mesh/Open3dTSDFfusion_mesh.ply.
2025-07-28 18:48:27.886 | INFO     | collab_data.gcs_utils:upload_file:179 - Uploading file /

### Redownload colmap files

In [35]:
colmap_fn = curated_dir + "environment/C0043/preproc/colmap/features.h5"
out_fn = "/workspace/fieldwork-data/birds/2024-02-06/environment/C0043/preproc/colmap/features.h5"

gcs_client.gcs.get_file(colmap_fn, out_fn)

ClientResponseError: 404, message='Not Found', url='https://storage.googleapis.com/download/storage/v1/b/fieldwork_processed/o/2024_02_06-session_0001environment%2FC0043%2Fpreproc%2Fcolmap%2Ffeatures.h5?alt=media'

In [ ]:

# file_path = gcs_client.glob(f"{all_buckets[-1]}/field/*final*")[0]